# Healthcare Chatbot

A chatbot is an intelligent piece of software that is capable of communicating and performing actions similar to a human. Chatbots are used a lot in customer interaction, marketing on social network sites and instantly messaging the client. 

In this Python project with source code, we are going to build a retrieval-based chatbot using deep learning techniques. A retrieval-based chatbot uses predefined input patterns and responses. It then uses some type of heuristic approach to select the appropriate response.  


The chatbot will be trained on the dataset which contains categories (intents), pattern and responses. We use a special recurrent neural network (LSTM) to classify which category the user’s message belongs to and then we will give a random response from the list of responses.
Let’s create a retrieval based chatbot using NLTK, Keras, Python, etc.

Here are the 5 steps to create a chatbot in Python from scratch:
1.	Import and load the data file
2.	Preprocess data
3.	Create training and testing data
4.	Build the model
5.	Create the GUI


### 1. Import and load the data file

We import the necessary packages for our chatbot and initialize the variables we will use in our Python project .

•	Tkinter- To create Graphical User interfaces (GUIs)

•	Keras- Keras is a neural networks library written in Python that is high-level in nature and makes it extremely simple and intuitive to use. It works as a wrapper to low-level libraries like TensorFlow or Theano high-level neural networks library.

•   SGD- class that implements the stochastic gradient descent optimizer an iterative method for optimizing an objective function with suitable smoothness properties.

•	Nltk- NLTK is a standard python library with prebuilt functions and utilities for the ease of use and implementation. It provides us various text processing libraries with a lot of test datasets.

•	WordNetLemmatizer -  Lemmatization in NLTK is the algorithmic process of finding the lemma of a word depending on its meaning and context.

•	Json- Normally the JSON functions are used to read and write directly from JSON files. The data file is in JSON format so we used the json package to parse the JSON file into Python.

•	Pickle- Pickling is a way to convert a python object (list, dict, etc.) into a character stream. 

•	Random- Random module is an in-built module of Python which is used to generate random numbers. It generates a random float uniformly in the semi-open range [0, 1)

•	NumPy- Stands for Numerical Python. Using NumPy, mathematical and logical operations on arrays can be performed. 


In [10]:
import nltk
nltk.download(['punkt','wordnet','omw-1.4'])
import tensorflow
import keras
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import random
import numpy as np

from tkinter import *
from keras.models import Sequential,load_model
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\speci\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\speci\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\speci\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


The data file is in JSON format so we used the json package to parse the JSON file into Python.

In [11]:
data_file = open('intents.json').read()
intents = json.loads(data_file)

### 2. Preprocess data

The first step is to tokenize the text data. Here we iterate through the patterns and tokenize the sentence using nltk.word_tokenize() function and append each word in the words list. We also create a list of classes for our tags.
The reason to tokenize is when we analyze a sentence our model will judge which of the following classes that is like greeting or goodbye...etc.


In [12]:
words=[]
classes=[]
documents=[]
ignore_words=['?','!','.']
for intent in intents['intents']:
	for pattern in intent['patterns']:
		#tokenize here
		w=nltk.word_tokenize(pattern)
		#print('Token is: {}'.format(w))
		words.extend(w)
		#(['hey', 'you'], 'greeting')
		documents.append((w, intent['tag']))
		# add the tag to classes list
		if intent['tag'] not in classes:
			classes.append(intent['tag'])
	
	# Final lists
	print('Words list is: {}'.format(words))
	print('Docs are: {}'.format(documents))
	print('Classes are: {}'.format(classes))

Words list is: ['Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hey', 'Hola', 'Hello', 'Good', 'day']
Docs are: [(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting')]
Classes are: ['greeting']
Words list is: ['Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hey', 'Hola', 'Hello', 'Good', 'day', 'Bye', 'See', 'you', 'later', 'Goodbye', 'Nice', 'chatting', 'to', 'you', ',', 'bye', 'Till', 'next', 'time']
Docs are: [(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 't

Now we will lemmatize each word and remove duplicate words from the list. Lemmatizing is the process of converting a word into its lemma form and then creating a pickle file to store the Python objects which we will use while predicting. 
We have to loop through these patterns for eg:-hi there how are you.
To look through that we also go for a single pattern in all the available patterns.

In [13]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = list(set(words))
classes = list(set(classes))
#print(words)
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

training = []
output_empty = [0]*len(classes)
# [0,0,0,0,0,0,0,0]


### 3. Create training and testing data

Now, we will create the training data in which we will provide the input and the output. Our input will be the pattern and output will be the class our input pattern belongs to. But the computer doesn’t understand text so we will convert text into numbers.

In [14]:

for doc in documents:
	bag = []
	pattern_words = doc[0]
	pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
	#print('Current Pattern Words: {}'.format(pattern_words))

	for w in words:
		bag.append(1) if w in pattern_words else bag.append(0)

	#print('Current Bag: {}'.format(bag))

	output_row = list(output_empty)
	output_row[classes.index(doc[1])] = 1
	#print('Current Output: {}'.format(output_row))

	training.append([bag, output_row])

#print('Training: {}'.format(training))
random.shuffle(training)
training = np.array(training,dtype=object)

In [15]:
train_x = list(training[:, 0])
train_y = list(training[:, 1])
print('X: {}'.format(train_x))
print('Y: {}'.format(train_y))

X: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

### 4. Build the model

We have our training data ready, now we will build a deep neural network that has 3 layers. We use the Keras sequential API for this. After training the model for 200 epochs, we achieved high accuracy on our model. Let us save the model as ‘chatbot_model.h5’.

In [16]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# compiling the model & define an optimizer function
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])

mfit = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', mfit)

Epoch 1/200
10/10 [==============================] - 14s 16ms/step - loss: 2.2620 - accuracy: 0.1064
Epoch 2/200
10/10 [==============================] - 0s 14ms/step - loss: 2.0861 - accuracy: 0.1915
Epoch 3/200
10/10 [==============================] - 0s 9ms/step - loss: 2.0313 - accuracy: 0.2128
Epoch 4/200
10/10 [==============================] - 0s 10ms/step - loss: 1.9932 - accuracy: 0.2553
Epoch 5/200
10/10 [==============================] - 0s 6ms/step - loss: 1.8538 - accuracy: 0.4043
Epoch 6/200
10/10 [==============================] - 0s 5ms/step - loss: 1.7374 - accuracy: 0.5106
Epoch 7/200
10/10 [==============================] - 0s 4ms/step - loss: 1.5321 - accuracy: 0.6170
Epoch 8/200
10/10 [==============================] - 0s 7ms/step - loss: 1.4057 - accuracy: 0.5745
Epoch 9/200
10/10 [==============================] - 0s 7ms/step - loss: 1.2881 - accuracy: 0.6383
Epoch 10/200
10/10 [==============================] - 0s 10ms/step - loss: 1.2268 - accuracy: 0.5957
Epoc

### 5. Create the GUI

To create GUI we use Tkinter. It is the standard Python interface to the Tk GUI toolkit, and is Python's de facto standard GUI..
The below code loads the dataset, the trained model, and the words and classes files .

In [17]:
intents = json.loads(open('intents.json').read())
model = load_model('chatbot_model.h5')
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

The bow function below is used to return a sentence after it has been processed i.e., tokenized, lemmentized and bagged.

In [18]:
def bow(sentence):
	sentence_words = nltk.word_tokenize(sentence)
	sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
	bag = [0]*len(words)
	for s in sentence_words:
		for i, w in enumerate(words):
			if w == s:
				bag[i]=1
	return (np.array(bag))

In the predict_class function the sentence entered by the user is tokenized and lemmatized by calling the bow function.
To generate an actual response, instead of returning the static text we'll use our prediction model that is created .
We create predict class and static class to make the model global ,so any function can access it.
Error threshold is defined and loop through, which is then used to convert the intent and probability into list so here we'll get the result having highest probability.


In [19]:
def predict_class(sentence):
	sentence_bag = bow(sentence)
	res = model.predict(np.array([sentence_bag]))[0]
	ERROR_THRESHOLD = 0.25
	results = [[i,r] for i,r in enumerate(res) if r > ERROR_THRESHOLD]
	#sort by probablity
	results.sort(key=lambda x: x[1], reverse=True)
	return_list = []
	for r in results:
		return_list.append({'intent':classes[r[0]], 'probablity':str(r[1])})
	return return_list

Result is returned after being processed and classified to the getResponse function. One of the values in the response will be a randomly returned on the basis of the appropriate tag that the model predicted.

In [20]:
def getResponse(ints):
	tag = ints[0]['intent']
	list_of_intents = intents['intents']
	for i in list_of_intents:
		if(i['tag']==tag):
			result=random.choice(i['responses'])
			break

	return result

The chatbot_response function calls the predict_class function to get the class with highest probability. The result is passed to the getResponse function and one of the response sentence under the particular class is returned.

In [21]:
def chatbot_response(msg):
	ints = predict_class(msg)
	res = getResponse(ints)
	return res

Let us first test our model before creating the chat GUI. We created a while loop which will run until the user stops entering any input.

In [22]:
loop=True
while(loop):
    user_msg=input("You: ")
    if user_msg=="":
        loop=False
    else:
        print("Chatbot: ",chatbot_response(user_msg))

1/1 [==============================] - 0s 129ms/step
Chatbot:  Hello, thanks for asking
1/1 [==============================] - 0s 19ms/step
Chatbot:  Hi there, how can I help?
1/1 [==============================] - 1s 743ms/step
Chatbot:  Please provide pharmacy name


Now, for the chat GUI to display the chat- the send function is created. It reads the message from the user and generates an actual response, instead of returning the static text, the prediction model would have to be used by calling the chatbot_response function.

In [23]:
def send():
	msg = TextEntryBox.get("1.0", 'end-1c').strip()
	TextEntryBox.delete('1.0', 'end')

	if msg != '':
		ChatHistory.config(state=NORMAL)
		ChatHistory.insert('end', "You: " + msg + "\n\n")

		res = chatbot_response(msg)
		ChatHistory.insert('end', "Bot: " + res + "\n\n")
		ChatHistory.config(state=DISABLED)
		ChatHistory.yview('end')

To add a chat history we add a text in base,we can add the border size,color,font...etc as in the code below.
It should be non-edited because it will be showing the history.
Send button should be added because when we type something like "how are you?" we need a button to proceed.
After creating a button we define which function to be called which is the send function.
The mainloop() is an infinite loop used to run the chatbot, it waits for the user to enter a message and click on the send button, and process the event as long as the window is not closed.

In [26]:
base = Tk()
base.title("Healthcare Chatbot")
base.geometry("400x500")
base.resizable(width=False, height=False)

#chat history textview
ChatHistory = Text(base, bd=0, bg='white', font='Arial')
ChatHistory.config(state=DISABLED)

SendButton = Button(base, font=('Arial', 12, 'bold'), 
	text="Send", bg="#dfdfdf", activebackground="#3e3e3e", fg="#ffffff", command=send)

TextEntryBox = Text(base, bd=0, bg='white', font='Arial')

ChatHistory.place(x=6, y=6, height=386, width=386)
TextEntryBox.place(x=128, y=400, height=80, width=265)
SendButton.place(x=6, y=400, height=80, width=125)

base.mainloop()

1/1 [==============================] - 0s 14ms/step
